# ДЗ по "Сбор и разметка данных (семинары)"
## Семинар 9. Инструменты разметки наборов данных
* Домашнее задание:
- Задача 1.
Выберите датасет, который имеет отношение к вашей области интересов или исследований. Датасет должен содержать неструктурированные данные, требующие разметки для решения конкретной задачи, например, анализа настроений или распознавания именованных сущностей.

- Задача 2.
Выполните разметку на основе правил (rule-based labeling) на подмножестве выбранного датасета. Разработайте и реализуйте набор правил или условий, которые позволят автоматически присваивать метки данным на основе определенных шаблонов или критериев.

- Задача 3.
Выполните разметку вручную отдельного подмножества выбранного датасета с помощью выбранного вами инструмента разметки.

- Задача 4.
Объедините данные, размеченные вручную, с данными, размеченными на основе правил. Объедините два подмножества размеченных данных в один набор данных, сохранив при этом соответствующую структуру и целостность.

- Задача 5.
Обучите модель машинного обучения, используя объединенный набор размеченных данных. Разделите датасет на обучающий и тестовый наборы и используйте обучающий набор для обучения модели.

- Задача 6.
Оценить эффективность обученной модели на тестовом датасете. Используйте подходящие метрики оценки. Интерпретируйте результаты и проанализируйте эффективность модели в решении задачи разметки.



In [1]:
# Устанавливаем пакеты, если они не установлены
# !pip install pandas
# !pip install nltk


In [2]:
import pandas as pd
import nltk
import csv

In [3]:
# Загружаем предобученную модель токенизации текста
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\uc201\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# Загрузим датасет с новостями с lenta.ru
reviews = pd.read_csv('lenta_ru.csv', engine='python', on_bad_lines='skip')

In [5]:
reviews

,text
0,Бои у Сопоцкина и Друскеник закончились отступ...
1,"Министерство народного просвещения, в виду про..."
2,"Штабс-капитан П. Н. Нестеров на днях, увидев в..."
3,Фотограф-корреспондент Daily Mirror рассказыва...
4,"Лица, приехавшие в Варшаву из Люблина, передаю..."
...,...
3995,Министр внутренних дел РФ Владимир Рушайло и н...
3996,"8 февраля, впервые после раскола, состоялось с..."
3997,Нью-йоркское издательство PublicAffairs намере...
3998,Первый заместитель министра внутренних дел Рос...


In [127]:
df_td = pd.read_csv('kartaslovsent.csv', sep=';', on_bad_lines='skip')

In [128]:
df_td

,term,tag,value,pstv,ngtv,neut,dunno,pstvNgtvDisagreementRatio
0,абажур,NEUT,0.08,0.185,0.037,0.580,0.198,0.00
1,аббатство,NEUT,0.10,0.192,0.038,0.578,0.192,0.00
2,аббревиатура,NEUT,0.08,0.196,0.000,0.630,0.174,0.00
3,абзац,NEUT,0.00,0.137,0.000,0.706,0.157,0.00
4,абиссинец,NEUT,0.28,0.151,0.113,0.245,0.491,0.19
...,...,...,...,...,...,...,...,...
46122,ёмкость,NEUT,0.00,0.167,0.000,0.690,0.143,0.00
46123,ёрзать,NGTV,-0.54,0.050,0.446,0.397,0.107,0.00
46124,ёрничать,NGTV,-0.79,0.078,0.529,0.236,0.157,0.00
46125,ёрш,NEUT,0.16,0.224,0.072,0.576,0.128,0.00


In [132]:
# Функция для присвоения метки настроения на основе наличия положительных или отрицательных слов
def get_sentiment(text, df_td):
    '''
    Функция вычисляет метрику настроения.

    :param text: текст который будем анализировать,
    :param df_td: тональный словарь,
    :return: возвращает оценку тональности: -0.35 (неготив) <= нейтрально >= (0.55) позитив 
    '''
    # Токенизация текста на отдельные слова
    words = nltk.word_tokenize(text.lower())
    count = 0      # Счетчик найденных слов
    sum_value = 0  # Сумма всех зачений весов найденных слов
    ret_value = 'NEUT' # Нейтральная оценка
    for word in words:
        if len(df_td[df_td.term == word]):
            count += 1
            df_filtered = df_td[df_td['term'] == word]
            value = df_filtered['value'].iloc[0]
            sum_value += value
    if count:
        sum_value = sum_value/count

    if sum_value < -0.35:
        ret_value = 'NGTV'  # Неготив
    elif sum_value > 0.55:
        ret_value = 'PSTV'  # Позитив
    
    return ret_value, sum_value   


In [133]:
words = get_sentiment(reviews['text'].iloc[0], df_td)


In [134]:
words

('NEUT', -0.07416666666666667)

In [123]:
count = 0      # Счетчик найденных слов
sum_value = 0  # Сумма всех зачений весов найденных слов
for word in words:
    if len(loyln[loyln.term == word]):
        count += 1
        df_filtered = loyln[loyln['term'] == word]
        value = df_filtered['value'].iloc[0]
        sum_value += value
if count:
    sum_value = sum_value/count

('неприятель', -1.0)
('участие', 0.59)
('огонь', 0.27)
('попытка', 0.43)
('пробиться', 0.68)
('шоссе', 0.02)
('перемешивание', 0.0)
('часть', 0.0)
('неприятельский', -1.0)
('отряд', 0.0)
('преследовать', -0.43)
('инвалид', -0.45)
sum_value/count =-0.07416666666666667


In [64]:
r

,term,tag,value,pstv,ngtv,neut,dunno,pstvNgtvDisagreementRatio


In [45]:
# # Датасет с ноаостями lenta.ru
# url = 'https://github.com/natasha/corus/blob/master/data/lenta-ru-news.csv.bz2'
# path = './lenta-ru-news.csv.bz2'
# # if not os.path.isfile("lenta-ru-news.csv.bz2"):
#     # Загружаем данные
# # wget.download(url, out=path)
# # Создаем генратор с новостями с lenta.ru
# records = load_lenta2(path)
# # Добаляем текст новостей в словарь
# news_dict = {'text': [next(records).text for _ in range(4_000)]}
# # next(records)

In [46]:
# len(news_dict['text'])

4000

In [55]:
# # Создаём датафрем
# df = pd.DataFrame(news_dict)
# df.to_csv('lenta_ru.csv', encoding='utf-8', index=False)


In [56]:
# len(news_dict['text'])

In [60]:
for _ in range(2000):
    next(records).text
    # print(item.text)